In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
import os

In [ ]:
# data dictionaries are [gain][frequency][inpt_power] = (output_power OR list of output powers)

def correct_for_attenuation(d, attenuation, min_input_power = None):
    for g in d:
        for f in d[g]:
            new_data = {}
            for inpt_pwr in d[g][f]:
                if min_input_power and (inpt_pwr < min_input_power):
                    continue
                new_data[inpt_pwr - attenuation] = d[g][f][inpt_pwr]
            d[g][f] = new_data

def merge_data(cal_data, d):
    for g in d:
        if not g in cal_data:
            cal_data[g] = {}
        for f in d[g]:
            if not f in cal_data[g]:
                cal_data[g][f] = {}
            for inpt_pwr in d[g][f]:
                if not inpt_pwr in cal_data[g][f]:
                    cal_data[g][f][inpt_pwr] = []
                cal_data[g][f][inpt_pwr].append(d[g][f][inpt_pwr])


In [ ]:
base_dir = "/media/thomas/Extreme SSD/orca_paper_data_files/power_calibration/b205/20210513-cal-data-2"
input_files_b205 = {
    os.path.join(base_dir, "power_calibration_data_10_atten_1.pickle"): 10,
    os.path.join(base_dir, "power_calibration_data_40_atten_1.pickle"): 40,
    os.path.join(base_dir, "power_calibration_data_70_atten_1.pickle"): 70,
    os.path.join(base_dir, "power_calibration_data_80_atten_1.pickle"): 80,
}
rigol_min_input_power = -25 # Minimum (uncorrect) input where where the signal generator has sufficient SNR for the measurement

input_files = {'/media/thomas/Extreme SSD/orca_paper_data_files/power_calibration/x310/20231208/power_calibration_data_dbA_rfsiggen.pickle': 0}
sig_gen_min_input_power = None

def load_cal_data(input_files, min_input_power):
    cal_data = {}
    for idx, path in enumerate(input_files):
        with open(path, 'rb') as f:
            d = pickle.load(f)
        if idx == len(input_files) - 1:
            min_input_power = None # No other data available, so don't enforce a minimum SNR for the highest attenuation measurement
        correct_for_attenuation(d, input_files[path], min_input_power=min_input_power)
        merge_data(cal_data, d)
    return cal_data

cal_data_b205 = load_cal_data(input_files_b205, rigol_min_input_power)[36] # Pick a single gain value to plot
cal_data_x310 = load_cal_data(input_files, sig_gen_min_input_power)[30]

freqs_to_plot = np.array([50e6, 450e6])

In [ ]:
matplotlib.rcParams.update({
        'font.size': 16,
        'legend.fontsize': 10,
        'lines.linewidth': 2,
    })

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), facecolor='white')

linestyles = ['-', ':']

for idx, freq in enumerate(freqs_to_plot):
    inpt_pwrs = list(cal_data_b205[freq].keys())
    inpt_pwrs.sort()
    ax.plot(inpt_pwrs, [np.mean(cal_data_b205[freq][x]) for x in inpt_pwrs], label = f'[B205] {freq/1000000} MHz', linestyle=linestyles[idx], color='C0')#, marker='o', markersize=4)

    inpt_pwrs = list(cal_data_x310[freq].keys())
    inpt_pwrs.sort()
    ax.plot(inpt_pwrs, [np.mean(cal_data_x310[freq][x]) for x in inpt_pwrs], label = f'[X310] {freq/1000000} MHz', linestyle=linestyles[idx], color='C1')#, marker='o', markersize=4)
ax.set_xlabel('Input Power [dBm]')
ax.set_ylabel('SDR Recorded Power [dBFS]')
ax.legend()
ax.grid()
ax.set_xlim(-110, -20)
fig.tight_layout()
#fig.savefig('power_calibration.png', dpi=300)
plt.show()